In [1]:
import numpy as np
import cv2
import time
from IPython.display import clear_output

def get_available_cameras (upper_bound = 10, lower_bound = 0):
    available = []
    
    for i in range (lower_bound, upper_bound):
        cap = cv2.VideoCapture (i)
    
        if (cap.isOpened ()):
            available.append (i)
    
        cap.release ()
    
    return available

get_available_cameras ()

[0]

In [2]:
cam1 = cv2.VideoCapture (0)

cv2.namedWindow ("frame", cv2.WINDOW_NORMAL)

cv2.resizeWindow ("frame", (960, 720))

while (True):
    ret1, frame1 = cam1.read ()
    
    cv2.waitKey (1)
    
    
    frame = frame1 + 100
    
    resized = cv2.resize (frame, (960, 720))
    
    cv2.imshow ("frame", resized)
    
    time.sleep (0.01)
    
    if (cv2.waitKey (1) & 0xFF == ord('q')):
        break

cam1.release ()

cv2.destroyAllWindows()